In [12]:
from google.colab import drive
from google.auth import default
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
%cd /content/drive/MyDrive/Coursework/STATS/315B/project

/content/drive/MyDrive/Coursework/STATS/315B/project


## Imports and Setup

In [88]:
## Project Imports
import gspread
import os
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import xlrd

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

In [68]:
creds, _ = default()
gc = gspread.authorize(creds)

worksheet = gc.open('full-data').sheet1
rows = worksheet.get_all_values()
full_data = pd.DataFrame.from_records(rows)

new_header = full_data.iloc[0]
full_data = full_data[1:]
full_data.columns = new_header 

In [69]:
full_data.head()

,State,Region,age18-29,age30-44,age45-64,age65+,Voting Population,voting_pop_1992,voting_pop_2000,voting_pop_2008,...,Hispanic,Asian + Other,pres1992,pres1996,pres2000,pres2004,pres2008,pres2012,pres2016,pres2020
1,Alabama,South,793864.00,910018.00,1287439.00,764162.00,3755483.00,0.545,0.501,0.584,...,0.021,0.022,0,0,0,0,0,0,0,0
2,Alaska,West,144391.00,146874.00,188064.00,72837.00,552166.00,0.636,0.65,0.643,...,0.081,0.139,0,0,0,0,0,0,0,0
3,Arizona,West,1138959.00,1288733.00,1657469.00,1120054.00,5205215.00,0.513,0.402,0.491,...,0.187,0.045,0,1,0,0,0,0,0,1
4,Arkansas,South,480811.00,558062.00,756882.00,477149.00,2272904.00,0.531,0.46,0.500,...,0.036,0.035,1,1,0,0,0,0,0,0
5,California,West,6937699.00,8020773.00,9876676.00,5188754.00,30023902.00,0.493,0.441,0.495,...,0.228,0.152,1,1,1,1,1,1,1,1


In [76]:
X = full_data.loc[:, full_data.columns != 'pres2020']
y = full_data.loc[:,full_data.columns == 'pres2020']

full_data.dtypes

X.drop("State", axis=1, inplace=True)
le = LabelEncoder()
region_enc = le.fit_transform(X['Region'])
X.drop("Region", axis=1, inplace=True)
X["Region"] = region_enc
cols = list(X)
X[cols] = pd.to_numeric(X[cols].stack(), errors='coerce').unstack()
y.head()
y = pd.to_numeric(y['pres2020'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/

In [86]:
X_test.head(11)

,age18-29,age30-44,age45-64,age65+,Voting Population,voting_pop_1992,voting_pop_2000,voting_pop_2008,Non-Educated Whites,White,...,Hispanic,Asian + Other,pres1992,pres1996,pres2000,pres2004,pres2008,pres2012,pres2016,Region
44,4794166.0,5683661.0,6554348.0,3225168.0,20257343.0,0.483,0.424,0.455,0.260,0.715,...,0.215,0.050,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
41,810642.0,905245.0,1293876.0,794795.0,3804558.0,0.448,0.457,0.553,0.339,0.892,...,0.034,0.013,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
47,1421944.0,1674181.0,2228053.0,1188393.0,6512571.0,0.527,0.509,0.618,0.269,0.887,...,0.055,0.057,0.0,0.0,0.0,0.0,1.0,1.0,1.0,2.0
13,265045.0,311202.0,402352.0,243494.0,1222093.0,0.638,0.535,0.586,0.558,1.144,...,0.070,0.032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
25,503817.0,560506.0,761461.0,439701.0,2265485.0,0.522,0.478,0.589,0.344,0.933,...,0.030,0.011,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
32,347092.0,381410.0,529294.0,330405.0,1588201.0,0.504,0.454,0.551,0.240,0.791,...,0.380,0.114,1.0,1.0,1.0,0.0,1.0,1.0,1.0,3.0
18,711676.0,839527.0,1189457.0,672765.0,3413425.0,0.530,0.504,0.557,0.486,1.043,...,0.026,0.025,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0
33,3439775.0,3877995.0,5302889.0,2964315.0,15584974.0,0.502,0.475,0.514,0.276,0.790,...,0.138,0.034,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
4,480811.0,558062.0,756882.0,477149.0,2272904.0,0.531,0.460,0.500,0.462,0.962,...,0.036,0.035,1.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0
31,1367084.0,1744872.0,2503610.0,1343626.0,6959192.0,0.557,0.501,0.582,0.309,0.891,...,0.111,0.054,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0


In [87]:
y_test.head(11)

44    0
41    0
47    1
13    0
25    0
32    1
18    0
33    1
4     0
31    1
14    1
Name: pres2020, dtype: int64

In [90]:
## Logistic Regression with Scaling
scaler = StandardScaler()
lr = LogisticRegression()
lrmodel = Pipeline([('standardize', scaler), ('log_reg', lr)])
lrmodel.fit(X_train,y_train)
lr_preds = forest.predict(X_test)
print(rf_preds)

[1 0 1 0 0 1 0 1 0 1 1]


In [81]:
## Random Forests
forest = RandomForestClassifier(n_estimators = 120, random_state = 83)
forest.fit(X_train,y_train)
rf_preds = forest.predict(X_test)
print(rf_preds)

[1 0 1 0 0 1 0 1 0 1 1]


In [83]:
## XGBoost
xgboost1 = xgb.XGBClassifier(base_score=0.5, booster='gbtree', random_state = 83, colsample_bytree = 0.3, learning_rate = 0.1, alpha = 10, n_estimators = 120)
xgboost1.fit(X_train,y_train)
xb_preds = xgboost1.predict(X_test)
print(xb_preds)

[1 0 1 0 0 1 0 1 0 1 1]


In [91]:
## Rerun with all of the above, but with a larger training set. Test over five examples.
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, test_size=0.1, random_state=32)

lrmodel.fit(X_train2, y_train2)
lr_preds2 = forest.predict(X_test2)
print(lr_preds2)

forest.fit(X_train2, y_train2)
rf_preds2 = forest.predict(X_test2)
print(rf_preds2)

xgboost1.fit(X_train2, y_train2)
xb_preds2 = xgboost1.predict(X_test2)
print(xb_preds2)

[0 0 1 0 1 1]
[0 0 1 0 1 1]
[0 0 1 0 1 1]


In [85]:
y_test2.head(6)

51    0
17    0
39    1
37    0
46    1
9     1
Name: pres2020, dtype: int64

In [93]:
importances = forest.feature_importances_
print(importances)

[0.02464499 0.01685043 0.01428929 0.02391884 0.02405431 0.02082301
 0.03827758 0.01339124 0.06748019 0.04464269 0.02416844 0.02175176
 0.04437511 0.02514717 0.01328882 0.08063692 0.11160229 0.08058314
 0.10654182 0.18869683 0.01483511]


In [95]:
ximportances = xgboost1.feature_importances_
print(ximportances)

[0.01224951 0.0119306  0.02732873 0.0436759  0.01331991 0.00834699
 0.03618351 0.0059474  0.01586057 0.0303529  0.01126212 0.00952028
 0.02960616 0.02782421 0.         0.10530068 0.17837496 0.00576605
 0.17360304 0.25354648 0.        ]


In [97]:
X_train.head()

,age18-29,age30-44,age45-64,age65+,Voting Population,voting_pop_1992,voting_pop_2000,voting_pop_2008,Non-Educated Whites,White,...,Hispanic,Asian + Other,pres1992,pres1996,pres2000,pres2004,pres2008,pres2012,pres2016,Region
9,159856.0,170079.0,147182.0,77004.0,554121.0,0.477,0.439,0.552,0.019,0.571,...,0.061,0.023,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0
50,918269.0,1061853.0,1594455.0,902134.0,4476711.0,0.682,0.645,0.693,0.455,1.148,...,0.043,0.033,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0
7,573324.0,651764.0,1034933.0,566806.0,2826827.0,0.641,0.566,0.604,0.265,0.869,...,0.069,0.034,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
48,1200139.0,1447060.0,1875264.0,1036046.0,5558509.0,0.598,0.562,0.603,0.364,0.967,...,0.108,0.123,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0
5,6937699.0,8020773.0,9876676.0,5188754.0,30023902.0,0.493,0.441,0.495,0.240,0.735,...,0.228,0.152,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0
